In [ ]:
import os
import sys

!git clone https://github.com/terryluan12/Genrify.git
!cd Genrify && make

sys.path.insert(0, "Genrify/src")

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

In [1]:
from datasources import download_datasets, convert_files
from preprocessing import preprocess
from cnn.testhandler.TestHandler import TestHandler

In [3]:
# Make sure to change subset_num to the subset which you are using
subset_num = 3 # not used for testing data
#Possible method values:
#"spec", "mel", "chroma", "mfcc" to retrieve exclusive training datasets
#"create_testing_data" to create testing data for the first time (not needed anymore due to option below)
#"test" to unzip already preprocessed testing data from Google Drive
wavTomp3Towav = True  #if testing out converting gtzan from wav to mp3 back to wav
method = "mfcc"

if method!="test":
    if method=="create_testing_data":
        !unzip /content/drive/MyDrive/"APS360 Team Project"/Test_Data/test_data_mp3.zip
        data_dir = "/content/test_data_mp3"
        output_dir = "/content/Genrify/src/datasources/test_data_wav"
        convert_files(data_dir, output_dir, wavToMp3=False)
    elif not os.path.isdir("Genrify/src/datasources/processed_data"):
        download_datasets("Genrify/src")
        if wavTomp3Towav:
            !mv /content/Genrify/src/datasources/Data/genres_original /content
            data_dir = "/content/genres_original"
            output1_dir = "/content/genres_original_mp3"
            convert_files(data_dir, output1_dir, wavToMp3=True)
            output2_dir = "/content/Genrify/src/datasources/Data/genres_original" #This will now contain the WAV files converted back from MP3
            convert_files(output1_dir, output2_dir, wavToMp3=False)

    preprocess(subset_num, method, "Genrify/src")

else:
    #Unzipping already preprocessed testing data
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_mel.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_spectrogram.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_chroma.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_mfcc.zip' -d '/'
    test_handler = TestHandler(batch_size=1, num_workers=1)

Downloaded Datasources


In [ ]:
from cnn import mfcc_model, mfcc_training
from utils import plot
from cnn.datahandler.DataHandler import DataHandler
import os
import torch

In [ ]:
root_dir = "/content/Genrify/src"
mfcc_data_dir = os.path.join(root_dir, "datasources", "mfcc")
mfcc_data_handler = DataHandler(mfcc_data_dir, batch_size=32, num_workers=4)
for ss, g in [(5, 0.3)]:
    for lr in [5e-5, 1e-4, 2e-4, 3e-4, 5e-4, 1e-3, 2e-3]:
        num_epochs = 60
        mfcc_cnn = mfcc_model.MFCC_Resnet(num_classes=10)
        print(f"Learning rate: {lr}, Subset size: {ss}, Gamma: {g}")
        best_epoch = mfcc_training.train(mfcc_cnn, mfcc_data_handler.train_loader, mfcc_data_handler.val_loader, num_epochs, lr, 32, ss, g, patience=5)
        model_path = mfcc_training.get_model_name(mfcc_cnn.name, 32, lr, best_epoch)
        %matplotlib inline
        # plot.plot_training_curve(model_path)
        # plot.plot_confusion_matrix(model_path, range(10))